# Exploration for Reported Assessment Results and Suppressed Assessment Information

## Imports and Such

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Import assessment data minus fully suppressed scores
assessments = pd.read_pickle('../data/school_based/assessments_clean.pkl')

# Import Suppressed outlier data
suppressed = pd.read_pickle ('../data/school_based/full_suppression.pkl')

In [8]:
# Look at pandas .shif and groupby to do year differences.  No muckery.

assessments

,year,system_name,school_name,test,subject,subject_area,student_group,valid_tests,pct_met_exceeded,system,...,school_type,magnet,charter,virtual,title_1,lat,long,locale,geometry,pct_not_met
0,2018,Achievement School District,Aspire Coleman,TNReady,ELA,English Language Arts,All Students,321.0,11.2,TN-00985,...,1-Regular school,2-No,1-Yes,NOTVIRTUAL,1-Yes,35.2141,-89.923641,11-City: Large,POINT (-89.92364 35.21410),88.8
1,2018,Achievement School District,Aspire Coleman,TNReady,ELA,English Language Arts,Black or African American,299.0,11.4,TN-00985,...,1-Regular school,2-No,1-Yes,NOTVIRTUAL,1-Yes,35.2141,-89.923641,11-City: Large,POINT (-89.92364 35.21410),88.6
2,2018,Achievement School District,Aspire Coleman,TNReady,ELA,English Language Arts,Black/Hispanic/Native American,312.0,11.5,TN-00985,...,1-Regular school,2-No,1-Yes,NOTVIRTUAL,1-Yes,35.2141,-89.923641,11-City: Large,POINT (-89.92364 35.21410),88.5
3,2018,Achievement School District,Aspire Coleman,TNReady,ELA,English Language Arts,Economically Disadvantaged,222.0,9.5,TN-00985,...,1-Regular school,2-No,1-Yes,NOTVIRTUAL,1-Yes,35.2141,-89.923641,11-City: Large,POINT (-89.92364 35.21410),90.5
4,2018,Achievement School District,Aspire Coleman,TNReady,ELA,English Language Arts,English Learners with Transitional 1-4,11.0,18.2,TN-00985,...,1-Regular school,2-No,1-Yes,NOTVIRTUAL,1-Yes,35.2141,-89.923641,11-City: Large,POINT (-89.92364 35.21410),81.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381475,2022,Wilson County,Wilson Central High School,EOC,Geometry,Math,Non-Economically Disadvantaged,324.0,43.2,TN-00950,...,1-Regular school,2-No,2-No,SUPPVIRTUAL,2-No,36.13409,-86.39511,41-Rural: Fringe,POINT (-86.39511 36.13409),56.8
381476,2022,Wilson County,Wilson Central High School,EOC,Geometry,Math,Non-English Learners/Transitional 1-4,331.0,43.2,TN-00950,...,1-Regular school,2-No,2-No,SUPPVIRTUAL,2-No,36.13409,-86.39511,41-Rural: Fringe,POINT (-86.39511 36.13409),56.8
381477,2022,Wilson County,Wilson Central High School,EOC,Geometry,Math,Non-Students with Disabilities,335.0,44.5,TN-00950,...,1-Regular school,2-No,2-No,SUPPVIRTUAL,2-No,36.13409,-86.39511,41-Rural: Fringe,POINT (-86.39511 36.13409),55.5
381479,2022,Wilson County,Wilson Central High School,EOC,Geometry,Math,Super Subgroup,129.0,20.2,TN-00950,...,1-Regular school,2-No,2-No,SUPPVIRTUAL,2-No,36.13409,-86.39511,41-Rural: Fringe,POINT (-86.39511 36.13409),79.8


## Listy McListface - A Place to look at the lists in my dataframe.

In [3]:
# Assessments Info
assessments.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 335452 entries, 0 to 381480
Data columns (total 25 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   year              335452 non-null  int64   
 1   system_name       335452 non-null  object  
 2   school_name       335452 non-null  object  
 3   test              335452 non-null  object  
 4   subject           335452 non-null  object  
 5   subject_area      335452 non-null  object  
 6   student_group     335452 non-null  object  
 7   valid_tests       335452 non-null  float64 
 8   pct_met_exceeded  335452 non-null  float64 
 9   system            335452 non-null  object  
 10  school            335452 non-null  object  
 11  school_lvl        335452 non-null  object  
 12  tot_enrolled      335452 non-null  object  
 13  fte_teachers      335452 non-null  object  
 14  stu_tchr_ratio    335452 non-null  object  
 15  school_type       335452 non-null  object  
 16 

In [4]:
# Student Groups
student_group_list = np.unique(assessments['student_group'].values).tolist()
student_group_list

['All Students',
 'American Indian or Alaska Native',
 'Asian',
 'Black or African American',
 'Black/Hispanic/Native American',
 'Economically Disadvantaged',
 'English Learner Transitional 1-4',
 'English Learners',
 'English Learners with Transitional 1-4',
 'Female',
 'Gifted',
 'Hispanic',
 'Male',
 'Native Hawaiian or Other Pacific Islander',
 'Non-Black/Hispanic/Native American',
 'Non-Economically Disadvantaged',
 'Non-English Learners/Transitional 1-4',
 'Non-Students with Disabilities',
 'Students with Disabilities',
 'Super Subgroup',
 'White']

In [5]:
# School Types
school_type_list = np.unique(assessments['school_type'].values).tolist()
school_type_list

['1-Regular school',
 '2-Special education school',
 '4-Alternative Education School']

In [6]:
# Subject Areas
subject_area_list = np.unique(assessments['subject_area'].values).tolist()
subject_area_list

['English Language Arts', 'Math', 'Science', 'Social Studies']

## Unsuppressed: Broad Overview of Results

### Overall Reported Proficiencies by School-Level, Subject Area, and Year.

In [7]:
# Subject Area Pivot Table
subject_area_pivot = pd.pivot_table(assessments,
                       values = 'pct_met_exceeded',
                       index = ['school_lvl', 'subject_area'],
                       columns = 'year',
                       aggfunc = np.mean)

# Get the current list of years
years = list(subject_area_pivot.columns)

# Include 2020 as nan values to show a gap when testing did not occur
years.insert(years.index(2021), 2020)

# Reindex and set the columns to years
subject_area_pivot = subject_area_pivot.reindex(columns=years)

# Reset index to convert multi-index into columns
subject_area_pivot.reset_index(inplace=True)

subject_area_pivot

year,school_lvl,subject_area,2018,2019,2020,2021,2022
0,Elementary,English Language Arts,31.335356,31.743917,NaN,28.813341,33.241191
1,Elementary,Math,35.167390,40.285962,NaN,30.810894,33.810746
2,Elementary,Science,51.924528,NaN,NaN,35.361688,36.740372
3,Elementary,Social Studies,33.868922,34.517535,NaN,32.873194,36.472918
4,High,English Language Arts,26.999677,33.137761,NaN,29.433537,34.868741
5,High,Math,23.140926,26.061050,NaN,20.137251,22.920209
6,High,Science,40.007669,NaN,NaN,34.933561,35.582858
7,High,Social Studies,27.042634,28.653125,NaN,30.545871,34.340348
8,Middle,English Language Arts,27.293836,28.934080,NaN,24.437352,29.489242
9,Middle,Math,36.256511,39.027643,NaN,34.114325,35.736341
